##Decision Tree Classifier

In [135]:
# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import tree
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

In [137]:
# Load dataset

dataset= pd.read_csv('dataset.csv')
#print(dataset.head()) # to see first lines of dataset


# Alocate features and labels

X = dataset.iloc[:, :-2]  # Features
y = dataset.iloc[:, -2:]  # Labels (last two columns)
print (X.shape)

# Perform one-hot encoding on the categorical features

X = pd.get_dummies(X, drop_first=True)
# drop_first argument is used to drop one of the columns for each categorical feature to avoid multicollinearity.

# Check if the target variable has more than 2 classes
if y.nunique().any() > 2:
    y = pd.get_dummies(y, drop_first=True)
    # Apply one-hot encoding to y to represent each class as a separate binary column.



(22083, 32)


In [138]:
# split the data into train, validation and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1) # train and validation as 75% and test as 25%
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # train as 75% and validation as 25%

print(X_train.shape, X_validation.shape, X_test.shape)
print(y_train.shape, y_validation.shape, y_test.shape)

(12421, 17429) (4141, 17429) (5521, 17429)
(12421, 2) (4141, 2) (5521, 2)


In [139]:
# Select and train model of classifier

clf = tree.DecisionTreeClassifier(splitter='random', random_state=1)
# evaluate the change of the classifier from gini to entropy?

clf.fit(X_train, y_train)

y_pred = clf.predict(X_train) 

In [140]:
# Evaluate the model

accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy: {accuracy}')

ValueError: multiclass-multioutput is not supported